# Cr₂ Potential Energy Surface Analysis

**Companion code for:** *On the Viability of Kohn-Sham Densities for Improved Coupled Cluster Accuracy*  
**Journal:** *Nat. Comm.*\
**Authors:** Abdulrahman Y. Zamani, Barbaro Zulueta, Andrew M. Ricciuti, John A. Keith, Kevin Carter-Fenk \
**DOI:** TBA [DOI]

---

## CCSD(T)@DFA Benchmark Analysis

**Supporting Information**

---

### Table of Contents

1. [Setup and Configuration](#1.-Setup-and-Configuration)
2. [Load Reference Data](#2.-Load-Reference-Data)
3. [ORCA Output Parsing and BDE Calculation](#3.-ORCA-Output-Parsing-and-BDE-Calculation)
4. [Error Statistics](#4.-Error-Statistics)
5. [Display Functions](#5.-Display-Functions)
6. [Results: CCSD(T)@DFA](#6.-Results:-CCSD(T)@DFA)
7. [Results: KS-DFT](#7.-Results:-KS-DFT)
8. [Results: ph-AFQMC](#8.-Results:-ph-AFQMC)
9. [Summary Statistics](#9.-Summary-Statistics)
10. [Data Export](#10.-Data-Export)
11. [Publication Figures](#11.-Publication-Figures)
12. [Spark Plots](#12.-Spark-Plots)

---

### Overview

This notebook computes and analyzes bond dissociation energies (BDEs) for 49 first-row transition metal diatomic molecules using:

- **CCSD(T)/CBS** with various DFT/HF reference orbitals (HF, SVWN5, PBE, PW91, R²SCAN, PBE0)
- **KS-DFT** methods (SVWN5, PBE, PW91, R²SCAN, B3LYP, PBE0, ωB97X-V, ωB97M-V)
- **ph-AFQMC** reference data (where available)

### Dataset

| Bond Type | Description | Count | ph-AFQMC |
|-----------|-------------|:-----:|:--------:|
| M–O | Metal oxides | 10 | ✓ |
| M–Cl | Metal chlorides | 9 | ✓ |
| M–H | Neutral metal hydrides | 10 | ✓ |
| M–H⁺ | Metal hydride cations | 10 | — |
| M–M | Homonuclear dimers | 10 | — |
| **Total** | | **49** | **29** |

### Computational Details

- **Basis set:** def2-TZVP (DFT geometry optimization), CBS(T/Q) extrapolation (CC single points)
- **Relativistic treatment:** X2C (all species except Mn₂), ZORA (Mn₂ and Mn with CC-PBE0)
- **Spin-orbit corrections:** Applied from experimental/theoretical references
- **Software:** ORCA 6.0

---

## 1. Setup and Configuration

### Dependencies

```
numpy>=1.20
pandas>=1.3
scipy>=1.7
matplotlib>=3.5
plotly>=5.0
```

Install with: `pip install numpy pandas scipy matplotlib plotly`

### Directory Structure

```
.
├── species/                      # ORCA output files for all species
│   ├── Sc-H/                     # Metal hydride
│   │   ├── Sc-H_pbe_opt_x2c.out           # DFT geometry optimization
│   │   ├── Sc-H_ccsdt_hf_cbs_x2c.out      # CCSD(T)@HF/CBS
│   │   ├── Sc-H_ccsdt_svwn5_cbs_x2c.out   # CCSD(T)@SVWN5/CBS
│   │   ├── Sc-H_ccsdt_pbe_cbs_x2c.out     # CCSD(T)@PBE/CBS
│   │   ├── Sc-H_ccsdt_pw91_cbs_x2c.out    # CCSD(T)@PW91/CBS
│   │   ├── Sc-H_ccsdt_r2scan_cbs_x2c.out  # CCSD(T)@R²SCAN/CBS
│   │   └── Sc-H_ccsdt_pbe0_cbs_x2c.out    # CCSD(T)@PBE0/CBS
│   ├── Sc-O/                     # Metal oxide
│   ├── Sc-Cl/                    # Metal chloride
│   ├── Sc-Sc/                    # Homonuclear dimer
│   ├── Sc-H+/                    # Metal hydride cation
│   ├── Sc/                       # Metal atom
│   ├── Sc+/                      # Metal cation
│   ├── H/                        # H atom (HF reference for CC)
│   ├── O/                        # O atom
│   ├── Cl/                       # Cl atom
│   └── ...                       # All 10 first-row TM species
│
├── ref_data/                     # Reference data
│   ├── species_ref_data.csv      # Experimental BDEs, spin-orbit corrections
│   └── ph-afqmc_data.csv         # ph-AFQMC reference values
│
└── generate_tables.py            # Standalone Excel export script
```

### File Naming Convention

- **DFT:** `{species}_{functional}_opt_x2c.out`
- **CC:** `{species}_ccsdt_{reference}_cbs_x2c.out`
- **Special:** `Mn-Mn_ccsdt_pbe0_cbs_zora.out` and `Mn_ccsdt_pbe0_cbs_zora.out` (ZORA for Mn₂ and Mn, respectively)

In [1]:
# =============================================================================
# Dependencies
# =============================================================================

from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Dict, List, Tuple
import numpy as np
import pandas as pd
from scipy import constants

# =============================================================================
# Configuration
# =============================================================================

ENERGY_UNIT = 'kcal/mol'  # Options: 'eV', 'kcal/mol', 'kJ/mol'
SPECIES_DIR = Path('species')
REF_DATA_FILE = Path('ref_data/species_ref_data.csv')
QMC_DATA_FILE = Path('ref_data/ph-afqmc_data.csv')

# =============================================================================
# Unit Conversion (exact values from scipy.constants)
# =============================================================================

_hartree_J = constants.value('Hartree energy')
_avogadro = constants.Avogadro
_calorie = constants.calorie

HARTREE_TO_KCAL = _hartree_J * _avogadro / (_calorie * 1000)
HARTREE_TO_EV = constants.value('Hartree energy in eV')
KCAL_TO_EV = HARTREE_TO_EV / HARTREE_TO_KCAL
KCAL_TO_KJ = _calorie

UNIT_CONVERSION = {'kcal/mol': 1.0, 'kJ/mol': KCAL_TO_KJ, 'eV': KCAL_TO_EV}
UNIT_PRECISION = {'kcal/mol': 2, 'kJ/mol': 2, 'eV': 2}

def convert_energy(value_kcal: float) -> float:
    """Convert energy from kcal/mol to selected unit."""
    return value_kcal * UNIT_CONVERSION[ENERGY_UNIT]

# =============================================================================
# Method Definitions
# =============================================================================

CC_METHODS = ['ccsdt_hf', 'ccsdt_svwn5', 'ccsdt_pbe', 'ccsdt_pw91', 'ccsdt_r2scan', 'ccsdt_pbe0']
DFT_METHODS = ['svwn5', 'pbe', 'pw91', 'r2scan', 'b3lyp', 'pbe0', 'wb97x-v', 'wb97m-v']
QMC_METHODS = ['qmc_mp2', 'qmc_cc', 'qmc_tz_qz']

METHOD_LABELS = {
    # CCSD(T) methods
    'ccsdt_hf': 'CC-HF', 'ccsdt_svwn5': 'CC-SVWN5', 'ccsdt_pbe': 'CC-PBE',
    'ccsdt_pw91': 'CC-PW91', 'ccsdt_r2scan': 'CC-R²SCAN', 'ccsdt_pbe0': 'CC-PBE0',
    # DFT methods
    'svwn5': 'SVWN5', 'pbe': 'PBE', 'pw91': 'PW91', 'r2scan': 'R²SCAN',
    'b3lyp': 'B3LYP', 'pbe0': 'PBE0', 'wb97x-v': 'ωB97X-V', 'wb97m-v': 'ωB97M-V',
}

QMC_LABELS = {'qmc_mp2': 'QMC-MP2', 'qmc_cc': 'QMC-CC', 'qmc_tz_qz': 'QMC-TZ/QZ'}

BOND_TYPES = ['M-H', 'M-H+', 'M-O', 'M-Cl', 'M-M']
QMC_BOND_TYPES = ['M-H', 'M-O', 'M-Cl']

print(f"Configuration: {ENERGY_UNIT}")
print(f"  Hartree → kcal/mol: {HARTREE_TO_KCAL:.10f}")
print(f"  Hartree → eV:       {HARTREE_TO_EV:.12f}")

Configuration: kcal/mol
  Hartree → kcal/mol: 627.5094740631
  Hartree → eV:       27.211386245988


---

## 2. Load Reference Data

In [2]:
# =============================================================================
# Experimental Reference Data
# =============================================================================

def classify_species(name: str) -> Tuple[str, List[str]]:
    """Classify species by bond type and determine constituent atoms."""
    name = name.strip()
    if name.endswith('+'):
        metal = name.replace('-H+', '')
        return 'M-H+', [f"{metal}+", 'H']
    parts = name.split('-')
    atom1, atom2 = parts[0], parts[1]
    if atom2 == 'H':
        return 'M-H', [atom1, 'H']
    elif atom2 == 'O':
        return 'M-O', [atom1, 'O']
    elif atom2 == 'Cl':
        return 'M-Cl', [atom1, 'Cl']
    elif atom1 == atom2:
        return 'M-M', [atom1, atom2]
    return 'Other', [atom1, atom2]

# Load experimental data
df_ref = pd.read_csv(REF_DATA_FILE)
df_ref['dimers'] = df_ref['dimers'].str.strip()
df_ref[['bond_type', 'atoms']] = df_ref['dimers'].apply(lambda x: pd.Series(classify_species(x)))
df_ref['E_SO'] = df_ref['E_SO'].fillna(0.0)
df_ref['uncertainty'] = df_ref['uncertainty'].fillna(0.0)

REF_DATA = df_ref.set_index('dimers').to_dict('index')
SPECIES_BY_TYPE = {bt: df_ref[df_ref['bond_type'] == bt]['dimers'].tolist() for bt in BOND_TYPES}

# =============================================================================
# ph-AFQMC Reference Data
# =============================================================================

df_qmc = pd.read_csv(QMC_DATA_FILE)
df_qmc['dimers'] = df_qmc['dimers'].str.strip()
df_qmc = df_qmc.rename(columns={
    'qmc_tz/qz': 'qmc_tz_qz',
    'qm_tz/qz_uncertainty': 'qmc_tz_qz_uncertainty',
    'qm_cc_uncertainty': 'qmc_cc_uncertainty',
})

QMC_DATA = {}
for _, row in df_qmc.iterrows():
    sp = row['dimers']
    QMC_DATA[sp] = {}
    for m in QMC_METHODS:
        val = row.get(m)
        unc = row.get(f'{m}_uncertainty')
        if pd.notna(val):
            QMC_DATA[sp][m] = (val, unc if pd.notna(unc) else 0.0)

# Summary
print("Reference Data Summary")
print("=" * 40)
print(f"{'Bond Type':<10} {'Exp':>8} {'ph-AFQMC':>10}")
print("-" * 40)
for bt in BOND_TYPES:
    n_exp = len(SPECIES_BY_TYPE[bt])
    n_qmc = sum(1 for sp in SPECIES_BY_TYPE[bt] if sp in QMC_DATA) if bt in QMC_BOND_TYPES else 0
    qmc_str = str(n_qmc) if n_qmc > 0 else '—'
    print(f"{bt:<10} {n_exp:>8} {qmc_str:>10}")
print("-" * 40)
print(f"{'Total':<10} {len(REF_DATA):>8} {len(QMC_DATA):>10}")

Reference Data Summary
Bond Type       Exp   ph-AFQMC
----------------------------------------
M-H              10         10
M-H+             10          —
M-O              10         10
M-Cl              9          9
M-M              10          —
----------------------------------------
Total            49         29


---

## 3. ORCA Output Parsing and BDE Calculation

In [3]:
# =============================================================================
# ORCA Output Parsing
# =============================================================================

def parse_cc_energy(path: Path) -> Optional[float]:
    """
    Parse CC energy from CBS extrapolation block.
    Looks for 'FINAL SINGLE POINT ENERGY' within 'Extrapolation of energy' section.
    """
    if not path.exists():
        return None
    energy = None
    in_extrapolation = False
    with open(path) as f:
        for line in f:
            if 'Extrapolation of energy' in line:
                in_extrapolation = True
            elif in_extrapolation and 'FINAL SINGLE POINT ENERGY' in line:
                energy = float(line.split()[-1])
                break
    return energy

def parse_dft_energy(path: Path) -> Optional[float]:
    """Parse DFT energy (last 'FINAL SINGLE POINT ENERGY' after geometry optimization)."""
    if not path.exists():
        return None
    energy = None
    with open(path) as f:
        for line in f:
            if 'FINAL SINGLE POINT ENERGY' in line:
                energy = float(line.split()[-1])
    return energy

def get_file_path(name: str, method: str, parent_species: str = None) -> Path:
    """
    Construct file path for ORCA output.

    Special cases:
    - H atom with CC methods: uses HF reference (H_hf_cbs_x2c.out)
    - Mn-Mn with CC-PBE0: uses ZORA relativistic treatment
    """
    is_cc = method.startswith('ccsdt')

    # H atom special case for CC methods
    if name == 'H' and is_cc:
        return SPECIES_DIR / 'H' / 'H_hf_cbs_x2c.out'

    # Determine relativistic treatment
    # Mn-Mn system uses ZORA with CC-PBE0; all others use X2C
    use_zora = (method == 'ccsdt_pbe0') and (parent_species == 'Mn-Mn' or name == 'Mn-Mn')
    rela = 'zora' if use_zora else 'x2c'

    # Basis set
    basis = 'cbs' if is_cc else 'tz'
    method_name = method

    return SPECIES_DIR / name / f"{name}_{method_name}_{basis}_{rela}.out"

# =============================================================================
# BDE Calculation
# =============================================================================

def calculate_bde(species: str, method: str) -> Optional[float]:
    """
    Calculate bond dissociation energy:
    D_e = E(atoms) - E(molecule) + E_SO

    Returns BDE in kcal/mol, or None if calculation fails.
    """
    is_cc = method.startswith('ccsdt')
    parse_func = parse_cc_energy if is_cc else parse_dft_energy

    # Get molecular energy
    mol_path = get_file_path(species, method, parent_species=species)
    e_mol = parse_func(mol_path)
    if e_mol is None:
        return None

    # Get atomic energies
    atoms = REF_DATA[species]['atoms']
    e_atoms = 0.0
    for atom in atoms:
        atom_path = get_file_path(atom, method, parent_species=species)
        e_atom = parse_func(atom_path)
        if e_atom is None:
            return None
        e_atoms += e_atom

    # Calculate BDE with spin-orbit correction
    e_so = REF_DATA[species]['E_SO']
    bde_hartree = e_atoms - e_mol
    bde_kcal = bde_hartree * HARTREE_TO_KCAL + e_so

    return bde_kcal

# =============================================================================
# Calculate All BDEs
# =============================================================================

cc_results: Dict[str, Dict[str, float]] = {}
dft_results: Dict[str, Dict[str, float]] = {}

for species in REF_DATA:
    cc_results[species] = {}
    dft_results[species] = {}

    for method in CC_METHODS:
        bde = calculate_bde(species, method)
        if bde is not None:
            cc_results[species][method] = bde

    for method in DFT_METHODS:
        bde = calculate_bde(species, method)
        if bde is not None:
            dft_results[species][method] = bde

# Count results
cc_count = sum(len(v) for v in cc_results.values())
dft_count = sum(len(v) for v in dft_results.values())
print(f"Calculated {cc_count} CCSD(T) and {dft_count} DFT bond dissociation energies.")

Calculated 294 CCSD(T) and 392 DFT bond dissociation energies.


---

## 4. Error Statistics

In [4]:
# =============================================================================
# Error Statistics
# =============================================================================

@dataclass
class ErrorStats:
    """Container for error statistics."""
    mae: float
    rmse: float
    max_err: float
    mse: float
    n: int

def compute_stats(calc: List[float], exp: List[float]) -> ErrorStats:
    """Compute error statistics from calculated and experimental values."""
    err = np.array(calc) - np.array(exp)
    return ErrorStats(
        mae=np.mean(np.abs(err)),
        rmse=np.sqrt(np.mean(err**2)),
        max_err=np.max(np.abs(err)),
        mse=np.mean(err),
        n=len(err)
    )

def stats_by_type(results: Dict, methods: List[str]) -> Dict[str, Dict[str, ErrorStats]]:
    """Compute statistics grouped by bond type."""
    stats = {bt: {} for bt in BOND_TYPES}
    for bt in BOND_TYPES:
        for m in methods:
            calc, exp = [], []
            for sp in SPECIES_BY_TYPE[bt]:
                bde_kcal = results.get(sp, {}).get(m)
                if bde_kcal is not None:
                    calc.append(convert_energy(bde_kcal))
                    exp.append(convert_energy(REF_DATA[sp]['D_e']))
            if calc:
                stats[bt][m] = compute_stats(calc, exp)
    return stats

def overall_stats(results: Dict, methods: List[str], include_best: bool = False) -> Dict[str, ErrorStats]:
    """Compute overall statistics, optionally including Best method."""
    stats = {}
    best_errors = []

    for m in methods:
        calc, exp = [], []
        for sp in REF_DATA:
            bde_kcal = results.get(sp, {}).get(m)
            if bde_kcal is not None:
                calc.append(convert_energy(bde_kcal))
                exp.append(convert_energy(REF_DATA[sp]['D_e']))
        if calc:
            stats[m] = compute_stats(calc, exp)

    if include_best:
        for sp in REF_DATA:
            exp_val = convert_energy(REF_DATA[sp]['D_e'])
            best_err = None
            for m in methods:
                bde_kcal = results.get(sp, {}).get(m)
                if bde_kcal is not None:
                    err = convert_energy(bde_kcal) - exp_val
                    if best_err is None or abs(err) < abs(best_err):
                        best_err = err
            if best_err is not None:
                best_errors.append(best_err)

        if best_errors:
            err_arr = np.array(best_errors)
            stats['cc_best'] = ErrorStats(
                mae=np.mean(np.abs(err_arr)),
                rmse=np.sqrt(np.mean(err_arr**2)),
                max_err=np.max(np.abs(err_arr)),
                mse=np.mean(err_arr),
                n=len(err_arr)
            )

    return stats

def qmc_overall_stats() -> Dict[str, ErrorStats]:
    """Compute overall statistics for QMC methods including QMC-Best."""
    stats = {}
    best_errors = []

    for m in QMC_METHODS:
        calc, exp = [], []
        for sp in QMC_DATA:
            if m in QMC_DATA[sp]:
                val_kcal, _ = QMC_DATA[sp][m]
                calc.append(convert_energy(val_kcal))
                exp.append(convert_energy(REF_DATA[sp]['D_e']))
        if calc:
            stats[m] = compute_stats(calc, exp)

    for sp in QMC_DATA:
        exp_val = convert_energy(REF_DATA[sp]['D_e'])
        best_err = None
        for m in QMC_METHODS:
            if m in QMC_DATA[sp]:
                val_kcal, _ = QMC_DATA[sp][m]
                err = convert_energy(val_kcal) - exp_val
                if best_err is None or abs(err) < abs(best_err):
                    best_err = err
        if best_err is not None:
            best_errors.append(best_err)

    if best_errors:
        err_arr = np.array(best_errors)
        stats['qmc_best'] = ErrorStats(
            mae=np.mean(np.abs(err_arr)),
            rmse=np.sqrt(np.mean(err_arr**2)),
            max_err=np.max(np.abs(err_arr)),
            mse=np.mean(err_arr),
            n=len(err_arr)
        )

    return stats

# Compute statistics
cc_stats = stats_by_type(cc_results, CC_METHODS)
dft_stats = stats_by_type(dft_results, DFT_METHODS)
cc_overall = overall_stats(cc_results, CC_METHODS, include_best=True)
dft_overall = overall_stats(dft_results, DFT_METHODS)

print("Error statistics computed.")

Error statistics computed.


---

## 5. Display Functions

In [5]:
# =============================================================================
# Display Utilities
# =============================================================================

# ANSI color codes
GREEN = "\033[92m"
BLUE = "\033[94m"
RESET = "\033[0m"

def fmt_val(val: float) -> str:
    """Format value with appropriate precision."""
    prec = UNIT_PRECISION[ENERGY_UNIT]
    return f"{val:.{prec}f}"

def fmt_err(val: float) -> str:
    """Format error with sign."""
    prec = UNIT_PRECISION[ENERGY_UNIT]
    return f"{val:+.{prec}f}"

def print_bde_table(results: Dict, methods: List[str], bond_type: str, is_cc: bool = False):
    """Print BDE comparison table with statistics and optional Best column."""
    species_list = SPECIES_BY_TYPE[bond_type]
    prec = UNIT_PRECISION[ENERGY_UNIT]
    col_w = 15

    header = f"{'Species':^10} {'Exp':^{col_w}}"
    for m in methods:
        header += f" {METHOD_LABELS[m]:^{col_w}}"
    if is_cc:
        header += f" {'CC-Best':^{col_w}}"

    print(f"\n{bond_type} ({ENERGY_UNIT})")
    print("=" * len(header))
    print(header)
    print("-" * len(header))

    errors = {m: [] for m in methods}
    if is_cc:
        errors['cc_best'] = []

    for sp in species_list:
        exp = convert_energy(REF_DATA[sp]['D_e'])
        sp_data = {}

        for m in methods:
            bde_kcal = results.get(sp, {}).get(m)
            if bde_kcal is not None:
                bde = convert_energy(bde_kcal)
                sp_data[m] = (bde, bde - exp)

        best_method = None
        if is_cc and sp_data:
            best_method = min(sp_data.keys(), key=lambda m: abs(sp_data[m][1]))
            errors['cc_best'].append(sp_data[best_method][1])

        row = f"{sp:^10} {fmt_val(exp):^{col_w}}"
        for m in methods:
            if m in sp_data:
                bde, err = sp_data[m]
                errors[m].append(err)
                cell = f"{fmt_val(bde)}({fmt_err(err)})"
                if is_cc and m == best_method:
                    row += f" {GREEN}{cell:^{col_w}}{RESET}"
                else:
                    row += f" {cell:^{col_w}}"
            else:
                row += f" {'—':^{col_w}}"

        if is_cc:
            if best_method:
                bde, err = sp_data[best_method]
                cell = f"{fmt_val(bde)}({fmt_err(err)})"
                row += f" {GREEN}{cell:^{col_w}}{RESET}"
            else:
                row += f" {'—':^{col_w}}"

        print(row)

    print("-" * len(header))
    stat_methods = methods + (['cc_best'] if is_cc else [])

    # Print statistics: RMSE, MAE, MAX, then N
    for stat_name, stat_func in [('RMSE', lambda e: np.sqrt(np.mean(np.array(e)**2))),
                                   ('MAE', lambda e: np.mean(np.abs(e))),
                                   ('MAX', lambda e: np.max(np.abs(e)))]:
        row = f"{stat_name:^10} {'':^{col_w}}"
        for m in stat_methods:
            if errors[m]:
                val = stat_func(errors[m])
                if m == 'cc_best':
                    row += f" {GREEN}{fmt_val(val):^{col_w}}{RESET}"
                else:
                    row += f" {fmt_val(val):^{col_w}}"
            else:
                row += f" {'—':^{col_w}}"
        print(row)

    # Print N (sample size) last with green highlight for cc_best
    row = f"{'N':^10} {'':^{col_w}}"
    for m in stat_methods:
        if errors[m]:
            n_val = f"{len(errors[m]):^{col_w}}"
            if m == 'cc_best':
                row += f" {GREEN}{n_val}{RESET}"
            else:
                row += f" {n_val}"
        else:
            row += f" {'—':^{col_w}}"
    print(row)

def print_qmc_table(bond_type: str):
    """Print ph-AFQMC BDE comparison table with QMC-Best column."""
    species_list = SPECIES_BY_TYPE[bond_type]
    col_w = 18

    header = f"{'Species':^10} {'Exp':^{col_w}}"
    for m in QMC_METHODS:
        header += f" {QMC_LABELS[m]:^{col_w}}"
    header += f" {'QMC-Best':^{col_w}}"

    print(f"\n{bond_type} ({ENERGY_UNIT}) — ph-AFQMC")
    print("=" * len(header))
    print(header)
    print("-" * len(header))

    errors = {m: [] for m in QMC_METHODS}
    errors['qmc_best'] = []

    for sp in species_list:
        if sp not in QMC_DATA:
            continue

        exp = convert_energy(REF_DATA[sp]['D_e'])
        sp_data = {}

        for m in QMC_METHODS:
            if m in QMC_DATA[sp]:
                val_kcal, unc_kcal = QMC_DATA[sp][m]
                val = convert_energy(val_kcal)
                unc = convert_energy(unc_kcal)
                sp_data[m] = (val, unc, val - exp)

        if not sp_data:
            continue

        best_method = min(sp_data.keys(), key=lambda m: abs(sp_data[m][2]))
        errors['qmc_best'].append(sp_data[best_method][2])

        row = f"{sp:^10} {fmt_val(exp):^{col_w}}"
        for m in QMC_METHODS:
            if m in sp_data:
                val, unc, err = sp_data[m]
                errors[m].append(err)
                cell = f"{fmt_val(val)}±{fmt_val(unc)}({fmt_err(err)})"
                if m == best_method:
                    row += f" {BLUE}{cell:^{col_w}}{RESET}"
                else:
                    row += f" {cell:^{col_w}}"
            else:
                row += f" {'—':^{col_w}}"

        val, unc, err = sp_data[best_method]
        cell = f"{fmt_val(val)}±{fmt_val(unc)}({fmt_err(err)})"
        row += f" {BLUE}{cell:^{col_w}}{RESET}"
        print(row)

    print("-" * len(header))

    # Print statistics: RMSE, MAE, MAX, then N
    for stat_name, stat_func in [('RMSE', lambda e: np.sqrt(np.mean(np.array(e)**2))),
                                   ('MAE', lambda e: np.mean(np.abs(e))),
                                   ('MAX', lambda e: np.max(np.abs(e)))]:
        row = f"{stat_name:^10} {'':^{col_w}}"
        for m in QMC_METHODS + ['qmc_best']:
            if errors[m]:
                val = stat_func(errors[m])
                if m == 'qmc_best':
                    row += f" {BLUE}{fmt_val(val):^{col_w}}{RESET}"
                else:
                    row += f" {fmt_val(val):^{col_w}}"
            else:
                row += f" {'—':^{col_w}}"
        print(row)

    # Print N (sample size) last with blue highlight for qmc_best
    row = f"{'N':^10} {'':^{col_w}}"
    for m in QMC_METHODS + ['qmc_best']:
        if errors[m]:
            n_val = f"{len(errors[m]):^{col_w}}"
            if m == 'qmc_best':
                row += f" {BLUE}{n_val}{RESET}"
            else:
                row += f" {n_val}"
        else:
            row += f" {'—':^{col_w}}"
    print(row)

---

## 6. Results: CCSD(T)@DFA

In [6]:
for bt in BOND_TYPES:
    print_bde_table(cc_results, CC_METHODS, bt, is_cc=True)


M-H (kcal/mol)
 Species         Exp            CC-HF         CC-SVWN5         CC-PBE          CC-PW91        CC-R²SCAN        CC-PBE0         CC-Best    
------------------------------------------------------------------------------------------------------------------------------------------
   Sc-H         50.00       53.97(+3.97)    55.12(+5.12)    55.06(+5.06)    55.03(+5.03)    44.44(-5.56)    44.57(-5.43)    53.97(+3.97)  
   Ti-H         47.40       45.58(-1.82)    43.37(-4.03)    43.49(-3.91)    43.47(-3.93)    45.76(-1.64)    45.81(-1.59)    45.81(-1.59)  
   V-H          56.10       59.37(+3.27)    54.60(-1.50)    59.75(+3.65)    59.74(+3.64)    59.64(+3.54)    59.62(+3.52)    54.60(-1.50)  
   Cr-H         51.10       54.78(+3.68)    55.90(+4.80)    55.88(+4.78)    55.87(+4.77)    55.86(+4.76)    55.87(+4.77)    54.78(+3.68)  
   Mn-H         41.10       40.25(-0.85)    40.32(-0.78)    40.38(-0.72)    40.32(-0.78)    40.35(-0.75)    40.34(-0.76)    40.38(-0.72)  
   Fe-H    

---

## 7. Results: KS-DFT

In [7]:
for bt in BOND_TYPES:
    print_bde_table(dft_results, DFT_METHODS, bt, is_cc=False)


M-H (kcal/mol)
 Species         Exp            SVWN5            PBE            PW91           R²SCAN           B3LYP           PBE0           ωB97X-V         ωB97M-V    
----------------------------------------------------------------------------------------------------------------------------------------------------------
   Sc-H         50.00       64.45(+14.45)   54.90(+4.90)    55.11(+5.11)    51.38(+1.38)    54.26(+4.26)    50.69(+0.69)    54.10(+4.10)    53.78(+3.78)  
   Ti-H         47.40       65.33(+17.93)   54.91(+7.51)    55.71(+8.31)    53.91(+6.51)    59.45(+12.05)   49.73(+2.33)    57.90(+10.50)   44.57(-2.83)  
   V-H          56.10       81.12(+25.02)   72.42(+16.32)   73.39(+17.29)   73.16(+17.06)   64.83(+8.73)    67.24(+11.14)   63.30(+7.20)    59.78(+3.68)  
   Cr-H         51.10       61.16(+10.06)   53.21(+2.11)    53.85(+2.75)    51.53(+0.43)    55.83(+4.73)    49.56(-1.54)    55.31(+4.21)    54.27(+3.17)  
   Mn-H         41.10       47.48(+6.38)    42.93(+1.8

---

## 8. Results: ph-AFQMC

In [8]:
for bt in QMC_BOND_TYPES:
    print_qmc_table(bt)


M-H (kcal/mol) — ph-AFQMC
 Species          Exp              QMC-MP2             QMC-CC           QMC-TZ/QZ           QMC-Best     
---------------------------------------------------------------------------------------------------------
   Sc-H          50.00        55.90±0.30(+5.90)  54.70±0.30(+4.70)  53.11±0.73(+3.11)  53.11±0.73(+3.11) 
   Ti-H          47.40        48.40±1.00(+1.00)  47.90±0.90(+0.50)          —          47.90±0.90(+0.50) 
   V-H           56.10        53.60±1.10(-2.50)          —                  —          53.60±1.10(-2.50) 
   Cr-H          51.10        54.80±0.50(+3.70)  51.00±0.40(-0.10)          —          51.00±0.40(-0.10) 
   Mn-H          41.10        41.70±1.90(+0.60)  39.40±1.20(-1.70)          —          41.70±1.90(+0.60) 
   Fe-H          44.80        42.70±1.70(-2.10)  41.70±1.70(-3.10)          —          42.70±1.70(-2.10) 
   Co-H          54.60        57.50±1.00(+2.90)  55.20±1.00(+0.60)  53.09±1.33(-1.51)  55.20±1.00(+0.60) 
   Ni-H          59

---

## 9. Summary Statistics

In [9]:
def print_summary():
    """Print overall summary with separate tables for comparable subsets."""
    prec = UNIT_PRECISION[ENERGY_UNIT]
    col_w = 12

    qmc_types = ['M-H', 'M-O', 'M-Cl']
    non_qmc_types = ['M-M', 'M-H+']

    def subset_stats(results, methods, bond_types, include_best=False, best_key='best'):
        stats = {}
        best_errors = []
        for m in methods:
            calc, exp = [], []
            for bt in bond_types:
                for sp in SPECIES_BY_TYPE[bt]:
                    bde_kcal = results.get(sp, {}).get(m)
                    if bde_kcal is not None:
                        calc.append(convert_energy(bde_kcal))
                        exp.append(convert_energy(REF_DATA[sp]['D_e']))
            if calc:
                stats[m] = compute_stats(calc, exp)

        if include_best:
            for bt in bond_types:
                for sp in SPECIES_BY_TYPE[bt]:
                    exp_val = convert_energy(REF_DATA[sp]['D_e'])
                    best_err = None
                    for m in methods:
                        bde_kcal = results.get(sp, {}).get(m)
                        if bde_kcal is not None:
                            err = convert_energy(bde_kcal) - exp_val
                            if best_err is None or abs(err) < abs(best_err):
                                best_err = err
                    if best_err is not None:
                        best_errors.append(best_err)
            if best_errors:
                err_arr = np.array(best_errors)
                stats[best_key] = ErrorStats(
                    mae=np.mean(np.abs(err_arr)), rmse=np.sqrt(np.mean(err_arr**2)),
                    max_err=np.max(np.abs(err_arr)), mse=np.mean(err_arr), n=len(err_arr)
                )
        return stats

    def qmc_subset_stats(bond_types):
        stats = {}
        best_errors = []
        for m in QMC_METHODS:
            calc, exp = [], []
            for bt in bond_types:
                for sp in SPECIES_BY_TYPE[bt]:
                    if sp in QMC_DATA and m in QMC_DATA[sp]:
                        val_kcal, _ = QMC_DATA[sp][m]
                        calc.append(convert_energy(val_kcal))
                        exp.append(convert_energy(REF_DATA[sp]['D_e']))
            if calc:
                stats[m] = compute_stats(calc, exp)

        for bt in bond_types:
            for sp in SPECIES_BY_TYPE[bt]:
                if sp not in QMC_DATA:
                    continue
                exp_val = convert_energy(REF_DATA[sp]['D_e'])
                best_err = None
                for m in QMC_METHODS:
                    if m in QMC_DATA[sp]:
                        val_kcal, _ = QMC_DATA[sp][m]
                        err = convert_energy(val_kcal) - exp_val
                        if best_err is None or abs(err) < abs(best_err):
                            best_err = err
                if best_err is not None:
                    best_errors.append(best_err)

        if best_errors:
            err_arr = np.array(best_errors)
            stats['qmc_best'] = ErrorStats(
                mae=np.mean(np.abs(err_arr)), rmse=np.sqrt(np.mean(err_arr**2)),
                max_err=np.max(np.abs(err_arr)), mse=np.mean(err_arr), n=len(err_arr)
            )
        return stats

    # Table 1: M-H, M-O, M-Cl (with ph-AFQMC comparison)
    print("\n" + "=" * 75)
    print(f"Overall Statistics: M-H, M-O, M-Cl ({ENERGY_UNIT})")
    print("=" * 75)
    print(f"{'Method':^14} {'RMSE':^{col_w}} {'MAE':^{col_w}} {'MAX':^{col_w}} {'N':^8}")
    print("-" * 75)

    cc_sub = subset_stats(cc_results, CC_METHODS, qmc_types, True, 'cc_best')
    dft_sub = subset_stats(dft_results, DFT_METHODS, qmc_types)
    qmc_sub = qmc_subset_stats(qmc_types)

    print("\nCCSD(T):")
    for m in CC_METHODS:
        if m in cc_sub:
            s = cc_sub[m]
            print(f"{METHOD_LABELS[m]:^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}")
    if 'cc_best' in cc_sub:
        s = cc_sub['cc_best']
        print(f"{GREEN}{'CC-Best':^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}{RESET}")

    print("\nDFT:")
    for m in DFT_METHODS:
        if m in dft_sub:
            s = dft_sub[m]
            print(f"{METHOD_LABELS[m]:^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}")

    print("\nph-AFQMC:")
    for m in QMC_METHODS:
        if m in qmc_sub:
            s = qmc_sub[m]
            print(f"{QMC_LABELS[m]:^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}")
    if 'qmc_best' in qmc_sub:
        s = qmc_sub['qmc_best']
        print(f"{BLUE}{'QMC-Best':^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}{RESET}")

    # Table 2: M-M, M-H+ (no ph-AFQMC)
    print("\n" + "=" * 75)
    print(f"Overall Statistics: M-M, M-H+ ({ENERGY_UNIT})")
    print("=" * 75)
    print(f"{'Method':^14} {'RMSE':^{col_w}} {'MAE':^{col_w}} {'MAX':^{col_w}} {'N':^8}")
    print("-" * 75)

    cc_sub2 = subset_stats(cc_results, CC_METHODS, non_qmc_types, True, 'cc_best')
    dft_sub2 = subset_stats(dft_results, DFT_METHODS, non_qmc_types)

    print("\nCCSD(T):")
    for m in CC_METHODS:
        if m in cc_sub2:
            s = cc_sub2[m]
            print(f"{METHOD_LABELS[m]:^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}")
    if 'cc_best' in cc_sub2:
        s = cc_sub2['cc_best']
        print(f"{GREEN}{'CC-Best':^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}{RESET}")

    print("\nDFT:")
    for m in DFT_METHODS:
        if m in dft_sub2:
            s = dft_sub2[m]
            print(f"{METHOD_LABELS[m]:^14} {s.rmse:^{col_w}.{prec}f} {s.mae:^{col_w}.{prec}f} {s.max_err:^{col_w}.{prec}f} {s.n:^8}")

print_summary()


Overall Statistics: M-H, M-O, M-Cl (kcal/mol)
    Method         RMSE         MAE          MAX         N    
---------------------------------------------------------------------------

CCSD(T):
    CC-HF         13.07         6.94        54.66        29   
   CC-SVWN5        8.28         5.24        25.91        29   
    CC-PBE         3.95         3.16         9.24        29   
   CC-PW91         3.92         3.16         9.03        29   
  CC-R²SCAN        4.34         3.23        12.43        29   
   CC-PBE0         4.47         3.44        11.58        29   
   CC-Best         2.01         1.53         5.06        29   

DFT:
    SVWN5         30.88        25.91        59.35        29   
     PBE          14.75        11.17        34.80        29   
     PW91         15.15        11.45        36.14        29   
    R²SCAN         9.11         7.14        21.64        29   
    B3LYP          6.67         5.32        14.13        29   
     PBE0          6.35         4.72      

---

## 10. Data Export

In [10]:
# =============================================================================
# Data Export
# =============================================================================
#
# For Excel export, use the standalone script: generate_tables.py
#
# Usage:
#     python generate_tables.py
#     # Select unit: 1 (eV), 2 (kcal/mol), 3 (kJ/mol)
#
# Output: bde_results_<unit>.xlsx with 5 sheets:
#     1. CC         - CCSD(T)@DFA results by bond type
#     2. KS-DFT     - DFT results by bond type
#     3. ph-AFQMC   - QMC results with uncertainties
#     4. Overall M-H, M-O, M-Cl - Statistics for QMC-comparable species
#     5. Overall M-H+, M-M      - Statistics for remaining species

print("For data export, use: python generate_tables.py")

For data export, use: python generate_tables.py


---

## 11. Publication Figures

In [11]:
# =============================================================================
# Plotting Setup
# =============================================================================

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

#  Plot settings
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans'],
    'font.size': 8,
    'axes.titlesize': 9,
    'axes.labelsize': 8,
    'axes.linewidth': 0.8,
    'pdf.fonttype': 42,
})

# Colorblind-safe palette
COLOR_DFA = "#0072B2"      # Blue
COLOR_CCSDT = "#D55E00"    # Vermillion
COLOR_PHAFQMC = "#009E73"  # Bluish green
KCAL_TO_EV = 1.0 / 23.06

def compute_mae_data():
    """Compute MAE statistics from results for plotting."""
    qmc_types = ['M-H', 'M-O', 'M-Cl']
    non_qmc_types = ['M-M', 'M-H+']

    def get_maes(results, methods, bond_types):
        maes = {bt: [] for bt in bond_types}
        maes['Overall'] = []
        for m in methods:
            for bt in bond_types:
                errs = [results.get(sp, {}).get(m, np.nan) - REF_DATA[sp]['D_e']
                        for sp in SPECIES_BY_TYPE[bt] if m in results.get(sp, {})]
                maes[bt].append(np.mean(np.abs(errs)) if errs else np.nan)
            all_errs = [results.get(sp, {}).get(m, np.nan) - REF_DATA[sp]['D_e']
                        for bt in bond_types for sp in SPECIES_BY_TYPE[bt] if m in results.get(sp, {})]
            maes['Overall'].append(np.mean(np.abs(all_errs)) if all_errs else np.nan)
        return maes

    def get_best_mae(results, methods, bond_types):
        maes = {bt: 0.0 for bt in bond_types}
        maes['Overall'] = 0.0
        for bt in bond_types:
            best_errs = []
            for sp in SPECIES_BY_TYPE[bt]:
                exp = REF_DATA[sp]['D_e']
                errs = [results.get(sp, {}).get(m) - exp for m in methods if m in results.get(sp, {})]
                if errs:
                    best_errs.append(min(errs, key=abs))
            maes[bt] = np.mean(np.abs(best_errs)) if best_errs else 0
        all_best = []
        for bt in bond_types:
            for sp in SPECIES_BY_TYPE[bt]:
                exp = REF_DATA[sp]['D_e']
                errs = [results.get(sp, {}).get(m) - exp for m in methods if m in results.get(sp, {})]
                if errs:
                    all_best.append(min(errs, key=abs))
        maes['Overall'] = np.mean(np.abs(all_best)) if all_best else 0
        return maes

    def get_qmc_best_mae(bond_types):
        maes = {bt: 0.0 for bt in bond_types}
        maes['Overall'] = 0.0
        for bt in bond_types:
            best_errs = []
            for sp in SPECIES_BY_TYPE[bt]:
                if sp not in QMC_DATA:
                    continue
                exp = REF_DATA[sp]['D_e']
                errs = [QMC_DATA[sp][m][0] - exp for m in QMC_METHODS if m in QMC_DATA[sp]]
                if errs:
                    best_errs.append(min(errs, key=abs))
            maes[bt] = np.mean(np.abs(best_errs)) if best_errs else 0
        all_best = []
        for bt in bond_types:
            for sp in SPECIES_BY_TYPE[bt]:
                if sp not in QMC_DATA:
                    continue
                exp = REF_DATA[sp]['D_e']
                errs = [QMC_DATA[sp][m][0] - exp for m in QMC_METHODS if m in QMC_DATA[sp]]
                if errs:
                    all_best.append(min(errs, key=abs))
        maes['Overall'] = np.mean(np.abs(all_best)) if all_best else 0
        return maes

    return {
        'fig1': {
            'dft': get_maes(dft_results, DFT_METHODS, qmc_types),
            'cc': get_maes(cc_results, CC_METHODS, qmc_types),
            'cc_best': get_best_mae(cc_results, CC_METHODS, qmc_types),
            'qmc_best': get_qmc_best_mae(qmc_types),
        },
        'fig2': {
            'dft': get_maes(dft_results, DFT_METHODS, non_qmc_types),
            'cc': get_maes(cc_results, CC_METHODS, non_qmc_types),
            'cc_best': get_best_mae(cc_results, CC_METHODS, non_qmc_types),
        }
    }

# Method labels for plots
dfa_labels = ['SVWN5', 'PBE', 'PW91', 'R²SCAN', 'PBE0', 'B3LYP', 'ωB97X-V', 'ωB97M-V']
ccsdt_labels = ['CC-HF', 'CC-SVWN5', 'CC-PBE', 'CC-PW91', 'CC-R²SCAN', 'CC-PBE0', 'CC-Best']
qmc_labels = ['QMC-Best']
chem_acc_ev = 1.0 / 23.06  # Chemical accuracy in eV

mae_data = compute_mae_data()
print("MAE data computed for figures.")

MAE data computed for figures.


In [12]:
# =============================================================================
# Interactive Figure 1: M-O, M-Cl, M-H, Overall (with ph-AFQMC)
# =============================================================================

bond_types_1 = ['M-O', 'M-Cl', 'M-H', 'Overall']

dfas_maes_1 = {bt: np.array(mae_data['fig1']['dft'][bt]) for bt in bond_types_1}
ccsdt_maes_1 = {bt: np.array(mae_data['fig1']['cc'][bt] + [mae_data['fig1']['cc_best'][bt]]) for bt in bond_types_1}
qmc_maes_1 = {bt: np.array([mae_data['fig1']['qmc_best'][bt]]) for bt in bond_types_1}

fig1 = make_subplots(rows=1, cols=4, subplot_titles=bond_types_1, horizontal_spacing=0.05)

for i, bt in enumerate(bond_types_1):
    col = i + 1
    dfa_vals = dfas_maes_1[bt] * KCAL_TO_EV
    cc_vals = ccsdt_maes_1[bt] * KCAL_TO_EV
    qmc_vals = qmc_maes_1[bt] * KCAL_TO_EV

    fig1.add_trace(go.Bar(x=dfa_labels, y=dfa_vals, name='KS-DFT' if i==0 else None,
        marker_color=COLOR_DFA, marker_line_color='black', marker_line_width=0.5,
        legendgroup='dft', showlegend=(i==0),
        hovertemplate='%{x}<br>MAE: %{y:.3f} eV<extra>KS-DFT</extra>'), row=1, col=col)

    fig1.add_trace(go.Bar(x=ccsdt_labels, y=cc_vals, name='CCSD(T)/CBS' if i==0 else None,
        marker_color=COLOR_CCSDT, marker_line_color='black', marker_line_width=0.5,
        legendgroup='cc', showlegend=(i==0),
        hovertemplate='%{x}<br>MAE: %{y:.3f} eV<extra>CCSD(T)/CBS</extra>'), row=1, col=col)

    fig1.add_trace(go.Bar(x=qmc_labels, y=qmc_vals, name='ph-AFQMC' if i==0 else None,
        marker_color=COLOR_PHAFQMC, marker_line_color='black', marker_line_width=0.5,
        legendgroup='qmc', showlegend=(i==0),
        hovertemplate='%{x}<br>MAE: %{y:.3f} eV<extra>ph-AFQMC</extra>'), row=1, col=col)

    fig1.add_hline(y=chem_acc_ev, line_dash="dash", line_color="black", line_width=1, row=1, col=col)

fig1.update_layout(
    title=dict(text='<b>Figure 1:</b> M-O, M-Cl, M-H Benchmark (with ph-AFQMC)', x=0.5, y=0.98),
    height=600, width=1400, barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.12, xanchor='center', x=0.5),
    template='plotly_white', margin=dict(b=150, t=100),
)
fig1.update_yaxes(title_text='MAE (eV)', row=1, col=1)
fig1.update_yaxes(range=[0, 2.1])
fig1.update_xaxes(tickangle=60)
fig1.show()

In [13]:
# =============================================================================
# Interactive Figure 2: M-H+, M-M, Overall (no ph-AFQMC)
# =============================================================================

bond_types_2 = ['M-H+', 'M-M', 'Overall']

dfas_maes_2 = {bt: np.array(mae_data['fig2']['dft'][bt]) for bt in bond_types_2}
ccsdt_maes_2 = {bt: np.array(mae_data['fig2']['cc'][bt] + [mae_data['fig2']['cc_best'][bt]]) for bt in bond_types_2}

fig2 = make_subplots(rows=1, cols=3, subplot_titles=['M-H⁺', 'M-M', 'Overall'], horizontal_spacing=0.06)

for i, bt in enumerate(bond_types_2):
    col = i + 1
    dfa_vals = dfas_maes_2[bt] * KCAL_TO_EV
    cc_vals = ccsdt_maes_2[bt] * KCAL_TO_EV

    fig2.add_trace(go.Bar(x=dfa_labels, y=dfa_vals, name='KS-DFT' if i==0 else None,
        marker_color=COLOR_DFA, marker_line_color='black', marker_line_width=0.5,
        legendgroup='dft', showlegend=(i==0),
        hovertemplate='%{x}<br>MAE: %{y:.3f} eV<extra>KS-DFT</extra>'), row=1, col=col)

    fig2.add_trace(go.Bar(x=ccsdt_labels, y=cc_vals, name='CCSD(T)/CBS' if i==0 else None,
        marker_color=COLOR_CCSDT, marker_line_color='black', marker_line_width=0.5,
        legendgroup='cc', showlegend=(i==0),
        hovertemplate='%{x}<br>MAE: %{y:.3f} eV<extra>CCSD(T)/CBS</extra>'), row=1, col=col)

    fig2.add_hline(y=chem_acc_ev, line_dash="dash", line_color="black", line_width=1, row=1, col=col)

fig2.update_layout(
    title=dict(text='<b>Figure 2:</b> M-H⁺, M-M Benchmark (CC & DFT only)', x=0.5, y=0.98),
    height=550, width=1000, barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.12, xanchor='center', x=0.5),
    template='plotly_white', margin=dict(b=120, t=100),
)
fig2.update_yaxes(title_text='MAE (eV)', row=1, col=1)
fig2.update_yaxes(range=[0, 1.8])
fig2.update_xaxes(tickangle=60)
fig2.show()

In [14]:
# =============================================================================
# Static Export: Publication-quality matplotlib figures with broken y-axis
# =============================================================================

def create_benchmark_figure_mpl(dfa_labs, ccsdt_labs, dfas_maes, ccsdt_maes, bond_types,
                                 output_name, phafqmc_labs=None, phafqmc_maes=None,
                                 panel_ylims=None, exclude_methods=None, figsize=None,
                                 break_ratio=(3, 2)):
    """
    Create publication-quality bar chart figure with optional broken y-axis per panel.

    Parameters:
        panel_ylims: dict mapping bond_type to either:
            - ((lower_min, lower_max), (upper_min, upper_max)) for broken axis
            - (y_min, y_max) for single axis (no divider)
        exclude_methods: list of method labels to exclude (e.g., ['SVWN5'])
        break_ratio: tuple (lower, upper) for height ratio of broken axis (default 3:2)
    """
    n_panels = len(bond_types)
    include_phafqmc = phafqmc_labs is not None and phafqmc_maes is not None
    if figsize is None:
        figsize = (11, 3.8) if n_panels == 4 else (8.5, 3.8)

    if exclude_methods is None:
        exclude_methods = []

    color_map = {"dfa": COLOR_DFA, "ccsdt": COLOR_CCSDT}
    if include_phafqmc:
        color_map["phafqmc"] = COLOR_PHAFQMC

    def build_series(bond_key):
        labels = []
        values = []
        colors = []

        # DFT methods (excluding specified ones)
        for j, lab in enumerate(dfa_labs):
            if lab not in exclude_methods:
                labels.append(lab)
                values.append(dfas_maes[bond_key][j])
                colors.append(color_map["dfa"])

        # CC methods (excluding specified ones)
        for j, lab in enumerate(ccsdt_labs):
            if lab not in exclude_methods:
                labels.append(lab)
                values.append(ccsdt_maes[bond_key][j])
                colors.append(color_map["ccsdt"])

        # QMC methods
        if include_phafqmc and bond_key in phafqmc_maes:
            for j, lab in enumerate(phafqmc_labs):
                labels.append(lab)
                values.append(phafqmc_maes[bond_key][j])
                colors.append(color_map["phafqmc"])

        return labels, np.array(values), colors

    # Create figure
    fig = plt.figure(figsize=figsize)

    chem_acc = 1.0 / 23.06
    panel_labels_list = ['a', 'b', 'c', 'd']

    # Store axes for break indicators
    all_axes_info = []  # List of (ax_lower, ax_upper, has_break) or (ax, None, False)

    bottom_start = 0.18
    total_height = 0.55

    for i, bond in enumerate(bond_types):
        # Get panel-specific y-limits
        ylims = panel_ylims[bond]

        # Position each panel manually
        panel_width = 0.18
        panel_left = 0.08 + i * 0.23

        # Check if this panel has a broken axis
        has_break = isinstance(ylims[0], tuple)

        if has_break:
            ylim_lower, ylim_upper = ylims
            # Use break_ratio parameter for lower:upper heights
            ratio_sum = break_ratio[0] + break_ratio[1]
            lower_height = total_height * break_ratio[0] / ratio_sum
            upper_height = total_height * break_ratio[1] / ratio_sum

            ax_lower = fig.add_axes([panel_left, bottom_start, panel_width, lower_height])
            ax_upper = fig.add_axes([panel_left, bottom_start + lower_height, panel_width, upper_height])

            all_axes_info.append((ax_lower, ax_upper, True))

            labels, vals, colors = build_series(bond)
            vals_ev = vals * KCAL_TO_EV
            x = np.arange(len(labels))

            # Plot bars on both axes
            ax_upper.bar(x, vals_ev, color=colors, edgecolor='black', linewidth=0.5, width=0.7)
            ax_lower.bar(x, vals_ev, color=colors, edgecolor='black', linewidth=0.5, width=0.7)

            # Chemical accuracy line on lower axis
            ax_lower.axhline(chem_acc, color='black', linestyle='--', linewidth=0.8, zorder=10)

            # Set y-limits
            ax_lower.set_ylim(ylim_lower)
            ax_upper.set_ylim(ylim_upper)

            # Calculate tick spacing based on ratio
            lower_range = ylim_lower[1] - ylim_lower[0]
            upper_range = ylim_upper[1] - ylim_upper[0]
            lower_tick = lower_range / break_ratio[0]
            upper_tick = upper_range / break_ratio[1]

            ax_lower.yaxis.set_major_locator(plt.MultipleLocator(lower_tick))
            ax_upper.yaxis.set_major_locator(plt.MultipleLocator(upper_tick))

            # Format tick labels - show all ticks on lower, hide bottom tick on upper
            ax_lower.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.2f}'))
            # Hide 0.90 (bottom tick of upper axis) to prevent overlap with 0.45
            upper_min = ylim_upper[0]
            def upper_fmt(x, p):
                if abs(x - upper_min) < 0.001:
                    return ''
                return f'{x:.2f}'
            ax_upper.yaxis.set_major_formatter(plt.FuncFormatter(upper_fmt))

            # Hide spines between axes
            ax_upper.spines['bottom'].set_visible(False)
            ax_lower.spines['top'].set_visible(False)
            ax_upper.spines['top'].set_visible(False)
            ax_upper.spines['right'].set_visible(False)
            ax_lower.spines['right'].set_visible(False)

            ax_upper.tick_params(bottom=False, labelbottom=False)
            ax_lower.tick_params(top=False)

            # Title
            ax_upper.set_title(bond, fontweight='bold', fontsize=9, pad=8)

            # Panel label
            ax_upper.text(-0.08, 1.15, panel_labels_list[i], transform=ax_upper.transAxes,
                          fontsize=11, fontweight='bold')

            # X-axis labels
            ax_lower.set_xticks(x)
            ax_lower.set_xticklabels(labels, rotation=60, ha='right', fontsize=6)

            # Y-axis labels
            ax_lower.tick_params(labelleft=True, labelsize=6)
            ax_upper.tick_params(labelleft=True, labelsize=6)

            # Grid
            ax_lower.yaxis.grid(True, linestyle='-', alpha=0.2, linewidth=0.5)
            ax_upper.yaxis.grid(True, linestyle='-', alpha=0.2, linewidth=0.5)
            ax_lower.set_axisbelow(True)
            ax_upper.set_axisbelow(True)

        else:
            # Single axis (no divider)
            ylim = ylims

            ax = fig.add_axes([panel_left, bottom_start, panel_width, total_height])
            all_axes_info.append((ax, None, False))

            labels, vals, colors = build_series(bond)
            vals_ev = vals * KCAL_TO_EV
            x = np.arange(len(labels))

            # Plot bars
            ax.bar(x, vals_ev, color=colors, edgecolor='black', linewidth=0.5, width=0.7)

            # Chemical accuracy line
            ax.axhline(chem_acc, color='black', linestyle='--', linewidth=0.8, zorder=10)

            # Set y-limits
            ax.set_ylim(ylim)

            # 5 intervals
            y_range = ylim[1] - ylim[0]
            tick_spacing = y_range / 5
            ax.yaxis.set_major_locator(plt.MultipleLocator(tick_spacing))
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.2f}'))

            # Hide spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

            # Title
            ax.set_title(bond, fontweight='bold', fontsize=9, pad=8)

            # Panel label
            ax.text(-0.08, 1.05, panel_labels_list[i], transform=ax.transAxes,
                    fontsize=11, fontweight='bold')

            # X-axis labels
            ax.set_xticks(x)
            ax.set_xticklabels(labels, rotation=60, ha='right', fontsize=6)

            # Y-axis labels
            ax.tick_params(labelleft=True, labelsize=6)

            # Grid
            ax.yaxis.grid(True, linestyle='-', alpha=0.2, linewidth=0.5)
            ax.set_axisbelow(True)

    # Y-axis label on left side
    fig.text(0.02, 0.45, 'MAE (eV)', va='center', ha='left',
            rotation='vertical', fontsize=9)

    # Draw break indicators (//) only for panels with breaks
    fig.canvas.draw()

    for i, (ax_lower, ax_upper, has_break) in enumerate(all_axes_info):
        if has_break:
            bbox_lower = ax_lower.get_position()

            center_x = bbox_lower.x0
            break_y = bbox_lower.y1  # At the break point

            dx = 0.003
            dy = 0.010
            gap = 0.003

            kwargs = dict(color='k', clip_on=False, linewidth=1.0, solid_capstyle='butt')

            # First diagonal line
            line1 = plt.Line2D([center_x - dx - gap/2, center_x + dx - gap/2],
                               [break_y - dy, break_y + dy],
                               transform=fig.transFigure, **kwargs)
            fig.add_artist(line1)

            # Second parallel diagonal line
            line2 = plt.Line2D([center_x - dx + gap/2, center_x + dx + gap/2],
                               [break_y - dy, break_y + dy],
                               transform=fig.transFigure, **kwargs)
            fig.add_artist(line2)

    # Legend
    legend_handles = [
        plt.Rectangle((0,0), 1, 1, facecolor=color_map["dfa"], edgecolor="black", linewidth=0.4, label="KS-DFT"),
        plt.Rectangle((0,0), 1, 1, facecolor=color_map["ccsdt"], edgecolor="black", linewidth=0.4, label="CCSD(T)/CBS"),
    ]
    if include_phafqmc:
        legend_handles.append(plt.Rectangle((0,0), 1, 1, facecolor=color_map["phafqmc"], edgecolor="black", linewidth=0.4, label="ph-AFQMC"))
    legend_handles.append(Line2D([0], [0], color='black', linestyle='--', linewidth=0.8, label="chemical accuracy"))

    # Center legend over plot area (depends on number of panels)
    legend_x = 0.08 + (n_panels - 1) * 0.23 / 2 + 0.09
    fig.legend(handles=legend_handles, loc='upper center', ncol=len(legend_handles),
               frameon=False, bbox_to_anchor=(legend_x, 0.96), fontsize=8, columnspacing=1.2)

    plt.savefig(f"{output_name}.pdf", dpi=300, bbox_inches='tight', facecolor='white')
    plt.savefig(f"{output_name}.png", dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()
    print(f"Saved: {output_name}.pdf and {output_name}.png")

# Generate Figure 1: M-O, M-Cl, M-H, Overall with ph-AFQMC
# SVWN5 excluded, mixed panel types
panel_ylims_fig1 = {
    'M-O': ((0, 0.45), (0.90, 0.95)),  # Broken axis (4:1 ratio)
    'M-Cl': (0, 0.32),                  # No divider, single axis
    'M-H': (0, 0.25),                   # No divider, single axis
    'Overall': (0, 0.50),               # No divider, single axis
}

create_benchmark_figure_mpl(
    dfa_labels,
    ccsdt_labels,
    dfas_maes_1,
    ccsdt_maes_1,
    ['M-O', 'M-Cl', 'M-H', 'Overall'],
    'fig_benchmark_bar_mpl_with_qmc',
    phafqmc_labs=qmc_labels,
    phafqmc_maes=qmc_maes_1,
    panel_ylims=panel_ylims_fig1,
    exclude_methods=['SVWN5']
)

# Generate Figure 2: M-H+, M-M, Overall without ph-AFQMC
# Exclude SVWN5 and CC-HF only, no broken axis needed
panel_ylims_fig2 = {
    'M-H+': (0, 0.25),
    'M-M': (0, 0.80),
    'Overall': (0, 0.55),
}

create_benchmark_figure_mpl(
    dfa_labels,
    ccsdt_labels,
    dfas_maes_2,
    ccsdt_maes_2,
    ['M-H+', 'M-M', 'Overall'],
    'fig_benchmark_bar_mpl_no_qmc',
    panel_ylims=panel_ylims_fig2,
    exclude_methods=['SVWN5', 'CC-HF'],
    figsize=(8.5, 3.8)
)

print("\nBar charts generated.")

Saved: fig_benchmark_bar_mpl_with_qmc.pdf and fig_benchmark_bar_mpl_with_qmc.png
Saved: fig_benchmark_bar_mpl_no_qmc.pdf and fig_benchmark_bar_mpl_no_qmc.png

Bar charts generated.


---

## 12. Spark Plots

In [15]:
# =============================================================================
# Interactive Spark Plots: Consolidated 5×2 matrix (Plotly)
# =============================================================================

def create_spark_plot_plotly():
    """Create interactive Plotly version of the spark plots."""
    from plotly.subplots import make_subplots

    # Bond types in order (consistent with bar plots)
    bond_types_order = ['M-O', 'M-Cl', 'M-H', 'M-H+', 'M-M']
    row_labels = ['a', 'b', 'c', 'd', 'e']

    # Method definitions
    cc_keys = CC_METHODS
    cc_labels = ['CC-HF', 'CC-SVWN5', 'CC-PBE', 'CC-PW91', 'CC-R²SCAN', 'CC-PBE0']

    dft_keys = DFT_METHODS
    dft_labels = ['SVWN5', 'PBE', 'PW91', 'R²SCAN', 'B3LYP', 'PBE0', 'ωB97X-V', 'ωB97M-V']

    # color schemes
    CC_COLORS = {
        'ccsdt_hf': '#EE6677', 'ccsdt_svwn5': '#228833', 'ccsdt_pbe': '#4477AA',
        'ccsdt_pw91': '#CCBB44', 'ccsdt_r2scan': '#66CCEE', 'ccsdt_pbe0': '#AA3377',
    }
    DFT_COLORS = {
        'svwn5': '#0077BB', 'pbe': '#33BBEE', 'pw91': '#009988', 'r2scan': '#EE7733',
        'b3lyp': '#CC3311', 'pbe0': '#EE3377', 'wb97x-v': '#BBBBBB', 'wb97m-v': '#332288',
    }

    def get_bde(sp, method):
        if method in CC_METHODS:
            return cc_results.get(sp, {}).get(method)
        else:
            return dft_results.get(sp, {}).get(method)

    # Create 5x2 subplot
    subplot_titles = []
    for bt in bond_types_order:
        subplot_titles.extend([f'{bt} - CCSD(T)/CBS', f'{bt} - KS-DFT'])

    fig = make_subplots(rows=5, cols=2, subplot_titles=subplot_titles,
                        vertical_spacing=0.08, horizontal_spacing=0.08)

    for row_idx, bond_type in enumerate(bond_types_order):
        species_list = SPECIES_BY_TYPE[bond_type]

        # Get experimental values and uncertainties
        exp_vals = [REF_DATA[sp]['D_e'] * KCAL_TO_EV for sp in species_list]
        uncs = [REF_DATA[sp]['uncertainty'] * KCAL_TO_EV for sp in species_list]

        # Left panel: CCSD(T)/CBS
        row = row_idx + 1

        # Experimental (hollow circles)
        fig.add_trace(go.Scatter(
            x=species_list, y=exp_vals, mode='lines+markers',
            name='Exp' if row_idx == 0 else None,
            legendgroup='exp', showlegend=(row_idx == 0),
            line=dict(color='black', width=1.5),
            marker=dict(size=8, color='white', line=dict(color='black', width=1.5)),
            error_y=dict(type='data', array=uncs, visible=True, color='black', thickness=1),
            hovertemplate='%{x}<br>Exp: %{y:.3f} eV<extra></extra>'
        ), row=row, col=1)

        # CC methods
        for method_key, method_label in zip(cc_keys, cc_labels):
            calc_vals = []
            for sp in species_list:
                bde = get_bde(sp, method_key)
                calc_vals.append(bde * KCAL_TO_EV if bde is not None else None)

            fig.add_trace(go.Scatter(
                x=species_list, y=calc_vals, mode='lines+markers',
                name=method_label if row_idx == 0 else None,
                legendgroup=method_key, showlegend=(row_idx == 0),
                line=dict(color=CC_COLORS[method_key], width=1.2),
                marker=dict(size=7, color=CC_COLORS[method_key]),
                hovertemplate=f'%{{x}}<br>{method_label}: %{{y:.3f}} eV<extra></extra>'
            ), row=row, col=1)

        # Right panel: KS-DFT
        # Experimental (hollow circles)
        fig.add_trace(go.Scatter(
            x=species_list, y=exp_vals, mode='lines+markers',
            name=None, legendgroup='exp', showlegend=False,
            line=dict(color='black', width=1.5),
            marker=dict(size=8, color='white', line=dict(color='black', width=1.5)),
            error_y=dict(type='data', array=uncs, visible=True, color='black', thickness=1),
            hovertemplate='%{x}<br>Exp: %{y:.3f} eV<extra></extra>'
        ), row=row, col=2)

        # DFT methods
        for method_key, method_label in zip(dft_keys, dft_labels):
            calc_vals = []
            for sp in species_list:
                bde = get_bde(sp, method_key)
                calc_vals.append(bde * KCAL_TO_EV if bde is not None else None)

            fig.add_trace(go.Scatter(
                x=species_list, y=calc_vals, mode='lines+markers',
                name=method_label if row_idx == 0 else None,
                legendgroup=method_key, showlegend=(row_idx == 0),
                line=dict(color=DFT_COLORS[method_key], width=1.2),
                marker=dict(size=7, color=DFT_COLORS[method_key]),
                hovertemplate=f'%{{x}}<br>{method_label}: %{{y:.3f}} eV<extra></extra>'
            ), row=row, col=2)

    fig.update_layout(
        title=dict(text='<b>BDE Comparison: CCSD(T)/CBS vs KS-DFT</b>', x=0.5, y=0.99),
        height=1400, width=1100,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5,
                    font=dict(size=9)),
        template='plotly_white',
        margin=dict(l=60, r=40, t=120, b=40),
    )

    # Update all y-axes
    for i in range(1, 6):
        fig.update_yaxes(title_text='BDE (eV)', row=i, col=1, title_font=dict(size=10))

    return fig

# Show interactive spark plot
spark_fig = create_spark_plot_plotly()
spark_fig.show()

In [16]:
# =============================================================================
# Spark Plots: Consolidated 5×2 matrix (all bond types in one figure)
# Nature Communications Supplementary Information
# =============================================================================

def create_consolidated_spark_plot():
    """
    Create Nature Communications SI quality spark plot as a 5×2 matrix.

    Layout:
        - 5 rows: M-O, M-Cl, M-H, M-H+, M-M (consistent with bar plots)
        - 2 columns: CCSD(T)/CBS (left), KS-DFT (right)
        - Column titles at top
        - Row labels (a, b, c, d, e) on left side
        - Legends outside top-right of row a subplots

    Color palettes: bright and vibrant schemes (colorblind-safe)
    """

    # Style settings
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
        'font.size': 8,
        'axes.linewidth': 0.8,
        'axes.labelsize': 9,
        'axes.titlesize': 10,
        'xtick.major.width': 0.8,
        'ytick.major.width': 0.8,
        'xtick.major.size': 4,
        'ytick.major.size': 4,
        'legend.fontsize': 6,
        'legend.frameon': True,
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'savefig.bbox': 'tight',
        'savefig.pad_inches': 0.1,
    })

    # Method definitions
    cc_keys = CC_METHODS
    cc_labels = ['CC-HF', 'CC-SVWN5', 'CC-PBE', 'CC-PW91', 'CC-R²SCAN', 'CC-PBE0']

    dft_keys = DFT_METHODS
    dft_labels = ['SVWN5', 'PBE', 'PW91', 'R²SCAN', 'B3LYP', 'PBE0', 'ωB97X-V', 'ωB97M-V']

    # Bright scheme for CC (6 colors)
    CC_COLORS = {
        'ccsdt_hf': '#EE6677',      # Red
        'ccsdt_svwn5': '#228833',   # Green
        'ccsdt_pbe': '#4477AA',     # Blue
        'ccsdt_pw91': '#CCBB44',    # Yellow
        'ccsdt_r2scan': '#66CCEE',  # Cyan
        'ccsdt_pbe0': '#AA3377',    # Purple
    }

    # Vibrant scheme for DFT (8 colors)
    DFT_COLORS = {
        'svwn5': '#0077BB',         # Blue
        'pbe': '#33BBEE',           # Cyan
        'pw91': '#009988',          # Teal
        'r2scan': '#EE7733',        # Orange
        'b3lyp': '#CC3311',         # Red
        'pbe0': '#EE3377',          # Magenta
        'wb97x-v': '#BBBBBB',       # Grey
        'wb97m-v': '#332288',       # Indigo
    }

    EXP_COLOR = '#000000'  # Black
    MARKER_SIZE = 25

    # Bond types in order for rows (consistent with bar plots)
    bond_types_order = ['M-O', 'M-Cl', 'M-H', 'M-H+', 'M-M']
    row_labels = ['a', 'b', 'c', 'd', 'e']

    def get_bde(sp, method):
        """Get BDE for species and method."""
        if method in CC_METHODS:
            return cc_results.get(sp, {}).get(method)
        else:
            return dft_results.get(sp, {}).get(method)

    # Figure size for 5×2 matrix
    fig_width = 14
    fig_height = 16
    fig, axes = plt.subplots(5, 2, figsize=(fig_width, fig_height))

    # Column titles
    fig.text(0.30, 0.98, 'CCSD(T)/CBS', ha='center', va='top', fontsize=12, fontweight='bold')
    fig.text(0.75, 0.98, 'KS-DFT', ha='center', va='top', fontsize=12, fontweight='bold')

    for row_idx, bond_type in enumerate(bond_types_order):
        species_list = SPECIES_BY_TYPE[bond_type]
        n_species = len(species_list)
        x_pos = np.arange(n_species)

        # Get experimental values and uncertainties
        exp_vals = np.array([REF_DATA[sp]['D_e'] * KCAL_TO_EV for sp in species_list])
        uncs = np.array([REF_DATA[sp]['uncertainty'] * KCAL_TO_EV for sp in species_list])

        # Left panel: CCSD(T)/CBS
        ax_cc = axes[row_idx, 0]

        # Plot experimental with error bars (hollow black circles)
        ax_cc.plot(x_pos, exp_vals, '-', c=EXP_COLOR, linewidth=1.2, zorder=9)
        for k, (x, exp, unc) in enumerate(zip(x_pos, exp_vals, uncs)):
            if unc > 0:
                ax_cc.errorbar(x, exp, yerr=unc, fmt='o', c=EXP_COLOR,
                              markerfacecolor='none', markersize=5, capsize=2,
                              capthick=0.6, elinewidth=0.6, zorder=10)
            else:
                ax_cc.scatter(x, exp, s=MARKER_SIZE, c='none', marker='o',
                             edgecolors=EXP_COLOR, linewidths=1.0, zorder=10)

        # Plot CC methods
        for method_key, method_label in zip(cc_keys, cc_labels):
            calc_vals = []
            for sp in species_list:
                bde = get_bde(sp, method_key)
                calc_vals.append(bde * KCAL_TO_EV if bde is not None else np.nan)
            calc_vals = np.array(calc_vals)
            color = CC_COLORS[method_key]
            ax_cc.plot(x_pos, calc_vals, '-', c=color, linewidth=1.0, alpha=0.9, zorder=4)
            ax_cc.scatter(x_pos, calc_vals, s=MARKER_SIZE, c=color, marker='o',
                         alpha=0.95, zorder=5, edgecolors='white', linewidths=0.3)

        # Styling for CC panel
        ax_cc.set_xlim(-0.5, n_species - 0.5)
        ax_cc.set_xticks(x_pos)
        ax_cc.set_xticklabels(species_list, rotation=45, ha='right', fontsize=7)
        ax_cc.set_ylabel('BDE (eV)', fontsize=9)
        ax_cc.spines['top'].set_visible(False)
        ax_cc.spines['right'].set_visible(False)
        ax_cc.tick_params(axis='both', labelsize=7)
        ax_cc.yaxis.grid(True, linestyle='-', alpha=0.2, linewidth=0.5)
        ax_cc.set_axisbelow(True)

        # Row label (a, b, c, d, e) on left side
        ax_cc.text(-0.15, 0.5, row_labels[row_idx], transform=ax_cc.transAxes,
                  fontsize=14, fontweight='bold', va='center', ha='right')

        # Right panel: KS-DFT
        ax_dft = axes[row_idx, 1]

        # Plot experimental with error bars (hollow black circles)
        ax_dft.plot(x_pos, exp_vals, '-', c=EXP_COLOR, linewidth=1.2, zorder=9)
        for k, (x, exp, unc) in enumerate(zip(x_pos, exp_vals, uncs)):
            if unc > 0:
                ax_dft.errorbar(x, exp, yerr=unc, fmt='o', c=EXP_COLOR,
                               markerfacecolor='none', markersize=5, capsize=2,
                               capthick=0.6, elinewidth=0.6, zorder=10)
            else:
                ax_dft.scatter(x, exp, s=MARKER_SIZE, c='none', marker='o',
                              edgecolors=EXP_COLOR, linewidths=1.0, zorder=10)

        # Plot DFT methods
        for method_key, method_label in zip(dft_keys, dft_labels):
            calc_vals = []
            for sp in species_list:
                bde = get_bde(sp, method_key)
                calc_vals.append(bde * KCAL_TO_EV if bde is not None else np.nan)
            calc_vals = np.array(calc_vals)
            color = DFT_COLORS[method_key]
            ax_dft.plot(x_pos, calc_vals, '-', c=color, linewidth=1.0, alpha=0.9, zorder=4)
            ax_dft.scatter(x_pos, calc_vals, s=MARKER_SIZE, c=color, marker='o',
                          alpha=0.95, zorder=5, edgecolors='white', linewidths=0.3)

        # Styling for DFT panel
        ax_dft.set_xlim(-0.5, n_species - 0.5)
        ax_dft.set_xticks(x_pos)
        ax_dft.set_xticklabels(species_list, rotation=45, ha='right', fontsize=7)
        ax_dft.spines['top'].set_visible(False)
        ax_dft.spines['right'].set_visible(False)
        ax_dft.tick_params(axis='both', labelsize=7)
        ax_dft.yaxis.grid(True, linestyle='-', alpha=0.2, linewidth=0.5)
        ax_dft.set_axisbelow(True)

    # Create legends outside top-right of row a subplots
    from matplotlib.lines import Line2D

    # CC legend handles (hollow circle for Exp)
    cc_handles = [Line2D([0], [0], marker='o', color=EXP_COLOR, markerfacecolor='none',
                         markersize=5, linewidth=1.2, label='Exp')]
    for method_key, method_label in zip(cc_keys, cc_labels):
        cc_handles.append(Line2D([0], [0], marker='o', color=CC_COLORS[method_key],
                                 markerfacecolor=CC_COLORS[method_key], markersize=5,
                                 linewidth=1.0, label=method_label))

    # DFT legend handles (hollow circle for Exp)
    dft_handles = [Line2D([0], [0], marker='o', color=EXP_COLOR, markerfacecolor='none',
                          markersize=5, linewidth=1.2, label='Exp')]
    for method_key, method_label in zip(dft_keys, dft_labels):
        dft_handles.append(Line2D([0], [0], marker='o', color=DFT_COLORS[method_key],
                                  markerfacecolor=DFT_COLORS[method_key], markersize=5,
                                  linewidth=1.0, label=method_label))

    # Add legends outside top-right of top row panels
    axes[0, 0].legend(handles=cc_handles, loc='best',
                      fontsize=6, framealpha=0.95, edgecolor='black', fancybox=False, ncol=1)
    axes[0, 1].legend(handles=dft_handles, loc='best',
                      fontsize=6, framealpha=0.95, edgecolor='black', fancybox=False, ncol=1)

    plt.tight_layout()
    plt.subplots_adjust(top=0.96, left=0.08, right=0.98, hspace=0.35, wspace=0.20)

    return fig

# Generate consolidated spark plot
fig = create_consolidated_spark_plot()
fig.savefig('spark_consolidated.pdf', dpi=300, bbox_inches='tight')
fig.savefig('spark_consolidated.png', dpi=300, bbox_inches='tight')
plt.close(fig)
print("Saved: spark_consolidated.pdf, spark_consolidated.png")

Saved: spark_consolidated.pdf, spark_consolidated.png


---

## References

1. Shee, J. et al. On Achieving High Accuracy in Quantum Chemical Calculations of 3d Transition Metal-Containing Systems: A Comparison of Auxiliary Field Quantum Monte Carlo with Coupled Cluster, Density Functional Theory, and Experiment for Diatomic Molecules *J. Chem. Theory Comput.* **2019**,2346.

2. Neese, F. et al. The ORCA quantum chemistry program package. *WIREs Comput. Mol. Sci.* **2025**, e70019.